In [1]:
# Shrirang Alias Samarth Patil
# 19BAI10079 

# !pip install Dataset
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq
# !pip install fsspec

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.8.2 requires cupy-cuda110, which is not installed.
cudf 21.8.2 requires cupy-cuda110, which is not installed.
s3fs 2021.8.1 requires fsspec==2021.08.1, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1.4.4 requires fsspec[http]!=2021.06.0,>=2021.05.0, but you have fsspec 2021.6.0 which is incompatible.
gcsfs 2021.7.0 requires fsspec==2021.07.0, but you have fsspec 2021.6.0 which is incompatible.
dask-cudf 21.8.2 requires dask<=2021.07.1,>=2021.6.0, but you have dask 2021.9.0 which is incompatible.
dask-cudf 21.8.2 requires distributed<=2021.07.1,>=2021.6.0, but you have distributed 2021.9.0 which is incompatible.


In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from transformers import *
from transformers import default_data_collator, Trainer
from transformers import AutoTokenizer, TrainingArguments,AutoModelForQuestionAnswering
import tensorflow as tf
# import collection
from datasets import Dataset
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


2021-12-18 15:42:00.969423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


/kaggle/input/hf-datasets/wheels/multiprocess-0.70.11.1-py37-none-any.whl
/kaggle/input/hf-datasets/wheels/tqdm-4.49.0-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/huggingface_hub-0.0.9-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/filelock-3.0.12-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/typing_extensions-3.10.0.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/certifi-2021.5.30-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/pandas-1.2.4-cp37-cp37m-manylinux1_x86_64.whl
/kaggle/input/hf-datasets/wheels/numpy-1.20.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
/kaggle/input/hf-datasets/wheels/fsspec-2021.6.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/datasets-1.7.0-py3-none-any.whl
/kaggle/input/hf-datasets/wheels/python_dateutil-2.8.1-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/idna-2.10-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/six-1.16.0-py2.py3-none-any.whl
/kaggle/input/hf-datasets/wheels/pyparsing-2.4.7-py2.p

In [3]:
train = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')
train.head()

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil


In [4]:
test = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
test.head()

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [5]:
tokenizer = AutoTokenizer.from_pretrained("../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2")

In [6]:
batch_size = 4
max_length = 384 
doc_stride = 128
pad_on_right = tokenizer.padding_side == "right"

In [7]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [8]:
def convert_answers(r):
    start = r[0]
    text = r[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train = train.sample(frac=1, random_state=42)
train['answers'] = train[['answer_start', 'answer_text']].apply(convert_answers, axis=1)

df_train = train[:-64].reset_index(drop=True)
df_valid = train[-64:].reset_index(drop=True)

train_dataset = Dataset.from_pandas(df_train)
valid_dataset = Dataset.from_pandas(df_valid)

In [9]:
train_dataset[0]

{'id': 'cd1339bdd',
 'context': 'एरीजोना (English: Arizona) संयुक्त राज्य अमेरिका के दक्षिणपश्चिमी हिस्से में स्थित राज्य है। इसका सबसे बड़ा शहर और राजधानी फ़िनक्स है। दूसरा सबसे बड़ा राज्य टक्सन है और उसके बाद फ़िनक्स के महानगर क्षेत्र स्थित शहर मेसा, ग्लेनडेल, चंदलर और स्कोत्ट्सडेल है।\nएरीजोना संयुक्त राज्यों में शामिल होने वाला ४८व राज्य था जिसे की इसमें १४ फरबरी १९१२ को समिलित किया गया। इसी दिन इस राज्य की देश में माने जाने वाली भूमि की वर्षागाठ भी थी। एरीजोना अपने रेगिस्तानी मौसम, भयंकर गर्मी और हल्की ठंड के लिए जाना जाता है। पर राज्य के उच्च इलाकों में बहुत पहाड़ व जंगल है, इसी कारण वहाँ का मौसम अन्य राज्य के मुकाबले ठंडा रहता है। जुलाई १ २००६ के अनुसार एरीजोना देश में आबादी के हिसाब से सबसे तेजी से बढ़ता राज्य है। इससे पहले यह दर्जा नेवादा के पास था।\nराज्य का एक-चौथाई इलाका मूल अमेरिकी आदिवासी के लिये संरक्षित है। राज्य की आधिकरिक भाषा अँग्रेजी है जिसे बोलने वाले जनसंख्या के 74 प्रतिशत है। स्पेनी 19.5 प्रतिशत द्वारा बोली जाती है। 2015 में राज्य की आबादी 68,28,065 अनुमानित की ग

In [10]:
tokenized_train_ds = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_valid_ds = valid_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [11]:
%env WANDB_DISABLED=True
args = TrainingArguments(
    f"chaii-qa",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    warmup_ratio=0.1,
    gradient_accumulation_steps=8,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


env: WANDB_DISABLED=True


In [12]:
data_collator = default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained('../input/xlm-roberta-squad2/deepset/xlm-roberta-large-squad2')

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [13]:
trainer.train()
trainer.save_model("chaii-bert-trained")

***** Running training *****
  Num examples = 13925
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 8
  Total optimization steps = 435


Epoch,Training Loss,Validation Loss
0,No log,0.264647


***** Running Evaluation *****
  Num examples = 816
  Batch size = 4
Saving model checkpoint to chaii-qa/checkpoint-435
Configuration saved in chaii-qa/checkpoint-435/config.json
Model weights saved in chaii-qa/checkpoint-435/pytorch_model.bin
tokenizer config file saved in chaii-qa/checkpoint-435/tokenizer_config.json
Special tokens file saved in chaii-qa/checkpoint-435/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to chaii-bert-trained
Configuration saved in chaii-bert-trained/config.json
Model weights saved in chaii-bert-trained/pytorch_model.bin
tokenizer config file saved in chaii-bert-trained/tokenizer_config.json
Special tokens file saved in chaii-bert-trained/special_tokens_map.json


In [14]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [15]:
validation_features = valid_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=valid_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
len(validation_features)

816

In [17]:
valid_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language', 'answers'],
    num_rows: 64
})

In [18]:
valid_feats_small = validation_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
valid_feats_small

  0%|          | 0/816 [00:00<?, ?ex/s]

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 816
})

In [19]:
raw_predictions = trainer.predict(valid_feats_small)

***** Running Prediction *****
  Num examples = 816
  Batch size = 4


In [20]:
raw_predictions[0]

(array([[  4.434763 ,  -6.9589934,  -8.717948 , ...,  -7.626959 ,
          -7.626959 ,  -7.626959 ],
        [  4.6128554,  -7.078187 ,  -8.950229 , ...,  -7.752864 ,
          -8.26884  ,  -9.282652 ],
        [  5.7324595,  -8.2656   ,  -9.081243 , ...,  -8.864551 ,
          -8.010219 ,  -9.165475 ],
        ...,
        [  5.549419 ,  -8.448947 ,  -7.5374217, ...,  -8.356556 ,
          -8.812152 ,  -9.627622 ],
        [  5.448872 ,  -8.389926 ,  -7.4379635, ...,  -8.549789 ,
          -7.3839283,  -9.540945 ],
        [  1.9692658,  -8.361491 , -10.233212 , ...,  -8.102062 ,
          -8.102062 ,  -8.102062 ]], dtype=float32),
 array([[  3.403032  ,  -8.955903  , -10.010152  , ...,  -8.9798155 ,
          -8.9798155 ,  -8.9798155 ],
        [  3.4916098 , -11.316827  , -10.7380495 , ...,  -9.975589  ,
          -7.4144464 , -10.602681  ],
        [  4.4809413 , -11.2317705 , -10.905625  , ...,  -8.687153  ,
          -9.393594  , -10.647553  ],
        ...,
        [  4.377657  

In [21]:
max_answer_length = 30

In [22]:
import collections

examples = valid_dataset
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [23]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        predictions[example["id"]] = best_answer["text"]

    return predictions

In [24]:
final_predictions = postprocess_qa_predictions(valid_dataset, validation_features, raw_predictions.predictions)

Post-processing 64 example predictions split into 816 features.


  0%|          | 0/64 [00:00<?, ?it/s]

In [25]:
prediction = pd.DataFrame([{"questions":x1['question'], "pred_answer":x2} for x1, x2 in zip(valid_dataset, [i for i in final_predictions.values()])])

In [26]:
prediction

,questions,pred_answer
0,राजस्थान की पहली महिला गवर्नर कौन थी?,प्रतिभा पाटील
1,புற்றுநோய் விழிப்புணர்வுக்கு என்ன நிறம்?,பிங்க்
2,भारतीय कवि मिर्ज़ा ग़ालिब का पूरा नाम क्या था?,मिर्ज़ा असद-उल्लाह बेग ख़ां
3,त्वरण की SI इकाई क्या है?,मीटर प्रति सेकेण्ड2
4,त्रिपुरा राज्य की राजधानी का नाम क्या है?,अगरतला
...,...,...
59,अरस्तु का जन्म कहाँ हुआ था?,स्टेगेरिया
60,சிந்து சமவெளி நாகரிகம் எப்போது உருவானது?,கி.மு 6000
61,आई एन एस विक्रमादित्य युद्धपोत किस कंपनी द्वार...,रूस
62,कितने अमेरिकी उपनिवेश अमेरिकी क्रांति का हिस्स...,तेरह


In [27]:
test_dataset = Dataset.from_pandas(test)

test_features = test_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=test_dataset.column_names
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [28]:
test_feats_small = test_features.map(lambda example: example, remove_columns=['example_id', 'offset_mapping'])
test_feats_small

  0%|          | 0/67 [00:00<?, ?ex/s]

Dataset({
    features: ['attention_mask', 'input_ids'],
    num_rows: 67
})

In [29]:
test_predictions = trainer.predict(test_feats_small)

***** Running Prediction *****
  Num examples = 67
  Batch size = 4


In [30]:
test_features.set_format(type=test_features.format["type"], columns=list(test_features.features.keys()))

In [31]:
final_test_predictions = postprocess_qa_predictions(test_dataset, test_features, test_predictions.predictions)

Post-processing 5 example predictions split into 67 features.


  0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
test['PredictionString'] = test['id'].apply(lambda r: final_test_predictions[r])
test.head()

,id,context,question,language,PredictionString
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi,येलन चीन
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi,14 जून 2006
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi,१२ मार्च १८२४
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil,13
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்
